In [19]:
import pandas
import numpy
import os
# from tqdm import tqdm_notebook

from xml.etree.ElementTree import parse, Element, SubElement, ElementTree
import xml.etree.ElementTree as ET

In [20]:
file_path = './2/labels'

In [21]:
#save_root1 = "./traffic_light"
save_root2 = "./traffic_sign"


#if not os.path.exists(save_root1):
#    os.mkdir(save_root1)

if not os.path.exists(save_root2):
    os.mkdir(save_root2)


def indent(elem, level=0): #자료 출처 https://goo.gl/J8VoDK
    i = "\n" + level*"    "
    if len(elem):
        if not elem.text or not elem.text.strip():
            elem.text = i + "    "
        if not elem.tail or not elem.tail.strip():
            elem.tail = i
        for elem in elem:
            indent(elem, level+1)
        if not elem.tail or not elem.tail.strip():
            elem.tail = i
    else:
        if level and (not elem.tail or not elem.tail.strip()):
            elem.tail = i

            
, filename, width, height, bbox_list):
    root = Element('annotation')
    SubElement(root, 'folder').text = folder
    SubElement(root, 'filename').text = filename
    SubElement(root, 'path').text = './images' +  filename
    source = SubElement(root, 'source')
    SubElement(source, 'database').text = 'Unknown'

    size = SubElement(root, 'size')
    SubElement(size, 'width').text = str(width)
    SubElement(size, 'height').text = str(height)
    SubElement(size, 'depth').text = '3'

    SubElement(root, 'segmented').text = '0'

    for i in bbox_list:
        obj = SubElement(root, 'object')
        SubElement(obj, 'name').text = i[0]
        SubElement(obj, 'pose').text = 'Unspecified'
        SubElement(obj, 'truncated').text = '0'
        SubElement(obj, 'difficult').text = '0'

        bbox = SubElement(obj, 'bndbox')
        SubElement(bbox, 'xmin').text = str(i[1])
        SubElement(bbox, 'ymin').text = str(i[2])
        SubElement(bbox, 'xmax').text = str(i[3])
        SubElement(bbox, 'ymax').text = str(i[4])

    indent(root)
    tree = ElementTree(root)
    tree.write('./'+folder + '/' + filename.split('.')[0] +'.xml')

In [22]:
#file_nm = glob.glob('./1/labels/')
file_nm = glob.glob('./2/labels/*.xml')
file_nm

['./2/labels/0711_13.xml',
 './2/labels/0704_05.xml',
 './2/labels/0704_25.xml',
 './2/labels/0708_10.xml',
 './2/labels/0704_06.xml',
 './2/labels/0708_11.xml',
 './2/labels/0709_17.xml',
 './2/labels/0711_14.xml',
 './2/labels/0704_04.xml',
 './2/labels/0710_01.xml',
 './2/labels/0704_10.xml',
 './2/labels/0709_08.xml',
 './2/labels/0709_01.xml',
 './2/labels/0710_28.xml',
 './2/labels/0709_06.xml',
 './2/labels/0704_20.xml',
 './2/labels/0710_06.xml',
 './2/labels/0704_12(200).xml',
 './2/labels/0710_11.xml',
 './2/labels/0708_14.xml',
 './2/labels/0708_01.xml',
 './2/labels/0710_12.xml',
 './2/labels/0710_03.xml',
 './2/labels/0709_13.xml',
 './2/labels/0708_17.xml',
 './2/labels/0710_04.xml',
 './2/labels/0710_25.xml',
 './2/labels/0704_02.xml',
 './2/labels/0708_16.xml',
 './2/labels/0711_15.xml',
 './2/labels/0708_12.xml',
 './2/labels/0708_05.xml',
 './2/labels/0710_30.xml',
 './2/labels/0704_13.xml',
 './2/labels/0708_15.xml',
 './2/labels/0710_02(1).xml',
 './2/labels/0704_14

In [23]:
for root_nm in file_nm:  # file_nm is xml_file_list
    before_xml = ET.parse(root_nm)
    root = before_xml.getroot()
    for child in root:
        if child.tag == 'image':
            file_nm = child.attrib['name']
            width = child.attrib['width']
            height = child.attrib['height']
            sign = 0
            bbox_list = []
            for i in child:
                image_box = []
                if (i.attrib['label'] == 'traffic_sign'):
                    sign += 1
                    image_box.append(i.attrib['label'])
                    image_box.append(i.attrib['xtl'])
                    image_box.append(i.attrib['ytl'])
                    image_box.append(i.attrib['xbr'])
                    image_box.append(i.attrib['ybr'])

                    bbox_list.append(image_box)
            if sign > 0:
                write_xml('./traffic_sign', file_nm, width, height, bbox_list)

In [24]:
import os
import glob
import pandas as pd
import xml.etree.ElementTree as ET


def xml_to_csv(path):
    xml_list = []
    for xml_file in glob.glob(path + '/*.xml'):
        tree = ET.parse(xml_file)
        root = tree.getroot()
        for member in root.findall('object'):
            value = (root.find('filename').text,
                     int(root.find('size')[0].text),
                     int(root.find('size')[1].text),
                     member[0].text,
                     member[4][0].text,
                     member[4][1].text,
                     member[4][2].text,
                     member[4][3].text
                     )
            xml_list.append(value)
    column_name = ['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax']
    xml_df = pd.DataFrame(xml_list, columns=column_name)
    return xml_df
    
labels_df = xml_to_csv('./2/traffic_sign/')
labels_df.to_csv(('./2/labels.csv'), index=None)
print('Successfully converted xml to csv.')

Successfully converted xml to csv.


In [25]:
# import libraries
import os
import shutil
import csv


# csv path
csv_path = os.path.join('./2', 'labels.csv')

# file root
root_path = os.path.join('./2/images')

# define the saving path after copying files by making new folder 
save_path = os.path.join('./2/traffic_images')
if not os.path.exists(save_path):
    os.mkdir(save_path)
#os.makedirs(save_path)

df = open(csv_path, 'r')
read = csv.reader(df, delimiter= ',')

flag = 0
for i, row in enumerate(read):
    if flag == 0:
        flag += 1
        pass
    else:
        type_id = row[3]
        dst = os.path.join(save_path, type_id)
        if not os.path.exists(dst):
            os.makedirs(dst)
        shutil.copy(os.path.join(root_path, row[0]), dst)  
